In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup
from tqdm import tqdm
from lxml import etree
from time import sleep
import tweepy
import schedule
import time
import pandas as pd
import requests
import pickle
import glob
import os
import joblib
import re
import warnings
warnings.simplefilter('ignore')

from python.get_title import get_title
from python.get_jockey_horse_detail import raceScrape_jockeyID_horseID
from python.main_scrape import mainScrape
from python.get_html import get_html_df
from python.get_race_detail import get_detail_df
from python.preprocessing import *
from python.get_html import tyoukyoushi
from python.create import create_shipping_time


DATA_DIR = '/pickle'
jockey_info = pd.read_pickle(DATA_DIR +'/jockey_late_start.pickle')

In [2]:
id = '202305050708'
# TODO G1などの値が取得できない(下のやつ)

In [10]:
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
options = Options() 
options.add_argument('--headless')
options.add_argument('--disable-gpu')             
options.add_argument('--disable-extensions')       
options.add_argument('--proxy-server="direct://"') 
options.add_argument('--proxy-bypass-list=*')      
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

raceDict = {}
timeList = []
# url = "https://race.netkeiba.com/top/"
url = "https://race.netkeiba.com/top/?kaisai_date=20231126"

driver.get(url)
sleep(1)

def getTime(date):
    soup = BeautifulSoup(table[0].get_attribute('outerHTML'), 'html.parser')
    sleep(1)
    raceDate = soup.select('.RaceList_Itemtime')
    if not raceDate:
        return []
    return raceDate[0].text.replace('[<span class="RaceList_Itemtime">', '').replace('\n</span>]', '')

# TODO 開催場が3つとは限らないため改善が必要
for j in tqdm(range(1,3)):
    for i in range(1,13):
        '''
        発走時刻の取得
        '''
        table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]/div[2]/div[2]/span[1]' )
        raceDate = getTime(table[0])
        
        # Win5の影響で取得するpathが変わるため
        if not raceDate:
            sleep(1)
            table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]/div[2]/div[2]/span[2]' )
            raceDate = getTime(table[0])
            
        # 末尾の空白削除
        raceDate = raceDate.rstrip()
        # 文字列をdatetimeオブジェクトに変換
        raceDate = datetime.strptime(raceDate, '%H:%M')
        # 20分前の時刻を計算
        raceDate = raceDate - timedelta(minutes=20)
        # 新しい時刻を文字列に変換
        raceDate = raceDate.strftime('%H:%M')
        timeList.append(raceDate)
        

        '''
        レースIDの取得
        '''
        sleep(1)
        table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]' )
        text = re.sub( r'\D+', '', str(table[0].get_attribute('href')))

        #[発走時刻 : レースID]の辞書型生成
        raceDict[raceDate] = text

        # 時刻を発走順にソート
        timeList.sort()
    

100%|██████████| 2/2 [01:01<00:00, 30.79s/it]


{'09:11': '202305050801',
 '09:35': '202305050802',
 '10:05': '202305050803',
 '10:35': '202305050804',
 '11:05': '202305050805',
 '11:55': '202305050806',
 '12:25': '202305050807',
 '12:55': '202305050808',
 '13:30': '202305050809',
 '14:05': '202305050810',
 '14:41': '202305050811',
 '15:20': '202305050812',
 '09:20': '202308030801',
 '09:50': '202308030802',
 '10:20': '202308030803',
 '10:50': '202308030804',
 '11:20': '202308030805',
 '12:10': '202308030806',
 '12:41': '202308030807',
 '13:10': '202308030808',
 '13:45': '202308030809',
 '14:21': '202308030810',
 '14:55': '202308030811',
 '15:55': '202308030812'}

In [3]:
def main()
    with open('./pkl/model.pkl', 'rb') as file:
        loaded_model = pickle.load(file)

    title = get_title(id)
    df = raceScrape_jockeyID_horseID(id)
    try:
        main = mainScrape(df)
    except:
        main = ''

    html_df = get_html_df(id)
    html_df = html_df[html_df['印'] != '取消']
    df = get_detail_df(df, id)
    df = df[df['印'] != '取消']
    horse_training = get_practice(html_df)
    df = prepro_race_name(df, title)
    df = df.drop(['印', '馬名', '厩舎', 'Unnamed: 9_level_0', 'お気に入り馬', '騎手'], axis=1)
    df = df.merge(horse_training, on='馬 番')
    df.loc[:, ['馬場', '脚色']] = df.loc[:, ['馬場', '脚色']].fillna('調教なし')
    df.loc[:, ['1C','2C','3C','4C','5C']] = df.loc[:, ['1C','2C','3C','4C','5C']].fillna(0)

    tyoukyoushi_list = joblib.load('./pkl/tyoukyoushi_list.pkl')
    df = tyoukyoushi(df, tyoukyoushi_list)
    df = change_tyoukyou_race_name(df)
    result = create_shipping_time(df)
    df = all_preprocessing(df, result, main)

    df = df.astype(float)
    pre =  pd.DataFrame(loaded_model.predict(df), columns=['predict'])
    pd.concat([df[['馬番']],pre],  axis=1)
    print_twitter(df, pre)

100%|██████████| 16/16 [01:02<00:00,  3.89s/it]


,馬番,predict
0,1.0,0
1,2.0,0
2,3.0,0
3,4.0,0
4,5.0,1
5,6.0,0
6,7.0,0
7,8.0,0
8,9.0,0
9,10.0,0


In [7]:
def print_twitter(df, pre):
    twwet_df = pd.concat([df[['馬番']],pre],  axis=1)
    twwet_df['馬番'] = twwet_df['馬番'].astype(int)
    t_list = twwet_df[twwet_df.predict == 1]['馬番'].to_list()
    text = '以下の馬番が出遅れます。\n'
        for i in t_list:
            text = text + str(i) + '番\n'

        text = text + '皆さん上記の馬には気を付けましょう。\n'

        tweet_text = text + '#' + title

    client = tweepy.Client(bearer_token=bearer_token,
                            consumer_key=consumer_key,
                            consumer_secret=consumer_secret,
                            access_token=access_token,
                            access_token_secret=access_token_secret)

    response = client.create_tweet(text=tweet_text)

In [ ]:
def task(task_id):
    try:
        print(f"タスク実行: {task_id}")
        main()
    except:
        print(f'エラー発生{task_id}')

for time_str, task_id in raceDict.items():
    schedule.every().day.at(time_str).do(task, task_id=task_id)

while True:
    schedule.run_pending()
    time.sleep(1)


In [ ]:
predict_parsent = pd.DataFrame(columns=['predict'])
for i in loaded_model.predict_proba(df):
    predict_parsent = pd.concat([predict_parsent, pd.DataFrame(i)])
predict_parsent = predict_parsent.loc[1][[0]]
predict_parsent = predict_parsent.reset_index().drop('index', axis=1).rename({0 : 'parsent'}, axis=1).sort_values(by='parsent', ascending=False)
predict_parsent.index = predict_parsent.index + 1
predict_parsent